In [123]:
"""
Python script to scrape Friktion User Data from Bitquery GraphQL API.

"""

import json
import requests
import requests
import traceback
import time

import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from os.path import exists


class MyPortfolio:
    """
    Python ETL script for Friktion user portfolio data. Currently supported Instruction Names:
        - Deposit
        - CancelPendingDeposit
        - Withdrawal
        - CancelPendingWithdrawal
        - ClaimPendingWithdrawal
    """

    def __init__(
        self,
        date_start,
        date_end,
        ix_fname="./friktion_ix.csv",
        deposit_fname="./friktion_deposit.csv",
        deposit_cxl_fname="./friktion_deposit_cxl.csv",
        withdraw_fname="./friktion_withdraw.csv",
        withdraw_cxl_fname="./friktion_withdraw_cancel.csv",
        withdraw_claim_fname="./friktion_claim_withdrawal.csv",
        batch_size_days=2,
        batch_size_xfers=90,
        skip_ix_scrape=False
    ):
        """
        :ix_fname:              output csv for instructions
        :deposit_fname:         output csv for deposits
        :deposit_cxl_fname:     output csv for deposit cancels
        :withdraw_fname:        output csv for withdrawals
        :withdraw_cxl_fname:    output csv for withdrawal cancels
        :withdraw_claim_fname:  output csv for claiming pending withdrawal
        :batch_size_days:       batch size in days for query to keep query < 10k symbols. Use bigger steps for larger data.
        :batch_size_transfers:  batch size transactions for query to keep query < 8kb

        """
        self.volt_program = "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"
        self.date_start = date_start
        self.date_end = date_end
        self.ix_fname = ix_fname
        self.deposit_fname = deposit_fname
        self.deposit_cxl_fname = deposit_cxl_fname
        self.withdraw_fname = withdraw_fname
        self.withdraw_cxl_fname = withdraw_cxl_fname
        self.withdraw_claim_fname = withdraw_claim_fname
        self.batch_size_days = batch_size_days
        self.batch_size_xfers = batch_size_xfers
        self.skip_ix_scrape = skip_ix_scrape
        
        self.df_ix = pd.read_csv(ix_fname) if skip_ix_scrape else [] 
        self.friktion_metadata = self.get_friktion_snapshot()

    ########################################################################################################
    ####################################          Queries             ######################################
    ########################################################################################################

    @property
    def ix_query(self):
        return """
            query MyQuery {
              solana {
                instructions(
                  time: {between: ["%s", "%s"]}
                  success: {is: true}
                  programId: {is: "VoLT1mJz1sbnxwq5Fv2SXjdVDgPXrb9tJyC8WpMDkSp"}
                  options: {limit: 8700}
                ) {
                  block {
                    timestamp {
                      iso8601
                    }
                  }
                  transaction {
                    signature
                    feePayer
                  }
                  data {
                    base58
                  }
                }
              }
            }
        """

    @property
    def xfer_query(self):
        return """
            query MyQuery {
              solana(network: solana) {
                transfers(
                  signature: {in: [%s]}
                ) {
                  instruction {
                    action {
                      name
                    }
                    callPath
                  }
                  amount(success: {is: true})
                  transaction {
                    signer
                    signature
                  }
                  block {
                    timestamp {
                      iso8601
                    }
                  }
                  currency {
                    name
                    address
                  }
                  sender {
                    address
                    mintAccount
                  }
                  receiver {
                    address
                  }
                }
              }
            }
        """

    ########################################################################################################
    ################################          Helper Functions             #################################
    ########################################################################################################

    # TODO: Add retry logic to this in case of hangups.
    @staticmethod
    def run_query(query, retries=10):
        """
        Query graphQL API.

        If timeerror
        """
        headers = {"X-API-KEY": "BQYCaXaMZlqZrPCSQVsiJrKtxKRVcSe4"}

        retries_counter = 0
        try:
            request = requests.post(
                "https://graphql.bitquery.io/", json={"query": query}, headers=headers
            )
            result = request.json()
            # print(dir(request.content))
            # Make sure that there is no error message
            # assert not request.content.errors
            assert "errors" not in result
        except:
            while (
                (request.status_code != 200
                or "errors" in result)
                and retries_counter < 10
            ):
                print(datetime.now(), f"Retry number {retries_counter}")
                if "errors" in result:
                    print(result["errors"])
                print(datetime.now(), f"Query failed for reason: {request.reason}. sleeping for {60*retries_counter} seconds and retrying...")
                time.sleep(120*retries_counter)
                request = requests.post(
                    "https://graphql.bitquery.io/",
                    json={"query": query},
                    headers=headers,
                )
                retries_counter += 1
            if retries_counter >= retries:
                raise Exception(
                    "Query failed after {} retries and return code is {}.{}".format(
                        retries_counter, request.status_code, query
                    )
                )
        return request.json()


    @staticmethod
    def batch_iterable(iterable, n=1):
        """
        Takes in an iterable and returns an iterable of iterables with len==x
        """
        idxs = []
        l = len(iterable)
        for idx in range(0, l, n):
            idxs.append(iterable[idx : min(idx + n, l)])
        return idxs

    def format_txs_for_query(self, tx_signatures):
        """
        Batches a list of transactions into a list of string formatted transactions for querying.
        Each of these strings contain (n=self.batch_size_xfers) unique transaction IDs.
        """
        batched_signatures = self.batch_iterable(tx_signatures, self.batch_size_xfers)

        def format_txs(x):
            return str(x)[1:-1].replace("'", '"').replace("\n", "")

        tx_strs = list(map(format_txs, batched_signatures))

        return tx_strs

    def get_existing_df(self, fname):
        """
        Create output file if doesn't exist and returns a DataFrame
        """
        if fname and exists(fname):
            return pd.read_csv(fname)
        else:
            return pd.DataFrame()

    @staticmethod
    def instruction_match(instructionData):
        """
        Match 8-bit instruction identifier with the corresonding instructionType
        """
        if not instructionData or len(instructionData) < 8:
            return False

        instructionDescriptor = instructionData[:8]

        if instructionDescriptor == "PcB3tF1K":
            return "Withdraw"
        elif instructionDescriptor == "WuE7Hjns":
            return "Deposit"
        elif instructionDescriptor == "V8cW2nMq":
            return "CancelPendingDeposit"
        elif instructionDescriptor == "dxUbSCWk":
            return "CancelPendingWithdrawal"
        elif instructionDescriptor == "WcTWQsnk":
            return "ClaimPendingWithdrawal"
        else:
            return "Unclassified"

    def get_friktion_snapshot(self):
        """
        Load Friktion Metadata for Volt/Symbol Mapping to join to normal data

        """
        try:
            return pd.DataFrame(
                dict(
                    json.loads(
                        requests.get(
                            "https://friktion-labs.github.io/mainnet-tvl-snapshots/friktionSnapshot.json"
                        ).content
                    )
                )["allMainnetVolts"]
            )[
                [
                    "globalId",
                    "vaultAuthority",
                    "shareTokenMint",
                    "depositTokenSymbol",
                    "depositTokenCoingeckoId",
                ]
            ]
        except Exception as e:
            print(datetime.now(), "Snapshot Data Invalid")
            traceback.print_exc()

    ########################################################################################################
    ################################          Data Retrieval             ###################################
    ########################################################################################################

    def get_ix(self, date_start, date_end):
        """
        Runs graphql instruction query for one date range.
        """
        print(date_start, date_end)
        query = self.ix_query % (date_start, date_end)
        print(
            datetime.now(),
            "retrieving instructions for {} to {}".format(date_start, date_end),
        )
        result = self.run_query(query)

        # convert GraphQL json to pandas dataframe
        try:
            df = pd.json_normalize(result["data"]["solana"]["instructions"])
        except:
            print(result)
            traceback.print_exc()
            raise Exception(datetime.now(), "Empty Results... Try Again")

        print(datetime.now(), df.shape[0], "instructions retrieved")

        df = df.rename(
            columns={
                "block.timestamp.iso8601": "timestamp",
                "log.consumed": "computeUnits",
                "transaction.signature": "txSignature",
                "transaction.feePayer": "txSigner",
                "data.base58": "instructionData",
            }
        )
        return df

    def get_ix_batch(self):
        """
        Batch the instruction retrieval. Save the shit Drop duplicates.

        """
        # Batch the days up nice and good so the graphql API calls don't bitch
        dates_batched = pd.date_range(self.date_start, self.date_end, freq=f"{self.batch_size_days}D")
        dates_batched = [
            str(x.isoformat())
            for x in dates_batched.append(pd.DatetimeIndex([self.date_end]))
        ]
        date_ranges = list(zip(dates_batched, dates_batched[1:]))
        ixs = []
        for date_range in date_ranges:
            assert len(date_range) == 2
            data = self.get_ix(date_range[0], date_range[1])
            ixs.append(data)

        df_ix = pd.concat(ixs, ignore_index=False)
        df_ix["instructionType"] = df_ix.instructionData.apply(
            lambda x: self.instruction_match(x)
        )

        # Store df_ix before we write it to the DataFrame so we avoid getting xfers for every single ix
        self.df_ix = df_ix.drop_duplicates(["txSignature", "instructionType"])
        print(datetime.now(), "final instruction data size: ", df_ix.shape[0])

        # Save new data to file
        df_old = self.get_existing_df(self.ix_fname)
        df = df_ix.append(df_old).reset_index(drop=True)
        df = df.drop_duplicates(["txSignature", "instructionType"])
        df.to_csv(self.ix_fname, index=False)
        print(datetime.now(), "wrote instruction data to csv...")

    def get_batched_xfers(self, instructionType, fname):
        """
        Get all transfers corresponding to a specific instructionType from Graphql query.
        Batch these queries up b/c the string size is
        too large (curse GraphQL for not supporting joins)

        :instructionType: String corresponding to the instruction type of each query.
        :fname: Name of where the old df is stored

        """

        temp = self.df_ix.query(f"instructionType == '{instructionType}'")

        if temp.empty:
            print(
                datetime.now(), "instructionType was not found in the data... breaking"
            )
            return

        tx_signatures = list(temp["txSignature"].unique())
        tx_strs = self.format_txs_for_query(tx_signatures)
        print(datetime.now(), len(tx_strs), "signature batches required...")
        xfers = []

        for i, tx_str in enumerate(tx_strs):
            query = self.xfer_query % (tx_str)
            result = self.run_query(query)
            # TODO: Clean up this duplicated logic
            try:
                df = pd.json_normalize(result["data"]["solana"]["transfers"])
            except:
                print(result)
                traceback.print_exc()
                raise Exception("Empty Results... Try Again")
            xfers.append(df)
            print(datetime.now(), df.shape[0], "transfers scraped in batch %d" % i)

        df_xfer = pd.concat(xfers, ignore_index=False)
        df_xfer = df_xfer.rename(
            columns={
                "block.timestamp.iso8601": "timestamp",
                "instruction.action.name": "instructionAction",
                "instruction.callPath": "instructionOrder",
                "transaction.signer": "userAddress",
                "transaction.signature": "txSignature",
                "currency.name": "currencyName",
                "receiver.address": "receiverAddress",
                "sender.address": "senderAddress",
                "currency.decimals": "currencyDecimals",
                "currency.address": "currencyAddress",
                "sender.mintAccount": "senderTokenMint",
            }
        )

        print(datetime.now(), df_xfer.shape[0], "transfers retrieved")

        df_old = self.get_existing_df(fname)
        df_final = df_old.append(df_xfer, ignore_index=True).sort_values(
            "instructionOrder"
        )

        return df_final

    def parse_deposits(self):
        instructionType = "Deposit"
        instructionAction = "transfer"
        tx_merge_key = "receiverAddress"
        meta_merge_key = "vaultAuthority"
        out_file = self.deposit_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_withdrawal(self):
        instructionType = "Withdraw"
        instructionAction = "burn"
        tx_merge_key = "currencyAddress"
        meta_merge_key = "shareTokenMint"
        out_file = self.withdraw_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_deposit_cancel(self):
        instructionType = "CancelPendingDeposit"
        instructionAction = "transfer"
        tx_merge_key = "senderAddress"
        meta_merge_key = "vaultAuthority"
        out_file = self.deposit_cxl_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_withdrawal_cancel(self):
        instructionType = "CancelPendingWithdrawal"
        instructionAction = "mintTo"
        tx_merge_key = "currencyAddress"
        meta_merge_key = "shareTokenMint"
        out_file = self.withdraw_cxl_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_claim_withdrawal(self):
        instructionType = "ClaimPendingWithdrawal"
        instructionAction = "transfer"
        tx_merge_key = "senderAddress"
        meta_merge_key = "vaultAuthority"
        out_file = self.withdraw_claim_fname

        self.parse_base(
            instructionType, instructionAction, tx_merge_key, meta_merge_key, out_file
        )

    def parse_base(
        self,
        instructionType,
        instructionAction,
        tx_merge_key,
        meta_merge_key,
        output_file,
    ):
        """
        generalized method for parsing transfer data.

        1. Call get_batched_xfers()
        2. for each unique txSignature, find the xfer matching to the last instance of the instructionAction
        3. For SOL CC vaults, need an extra step and query for instructionAccount to find vaultAuthority.
        3. Join it to the friktion metadata based using tx_merge_key and meta_merge_key
        4. Drop extraneous rows
        5. Save the file to the output_file

        :instructionType: Type of instruction listed out in the instruction_match() method
        :instructionAction: type of transfer we are matching towards
        :tx_merge_key: what key in the xfer dataFrame do we want to merge on
        :meta_merge_key: what key in the metadata dataFrame we want to merge on.
        :output_file: as name suggests
        """
        print(
            datetime.now(),
            "Parsing transfers for instructionType: %s" % instructionType,
        )
        df = self.get_batched_xfers(instructionType, output_file)

        # Target only wrapped SOL entries for SOL vaults
        df = df.query('currencyName != "Solana"')

        df = (
            df.query('instructionAction=="{}"'.format(instructionAction))
            .groupby("txSignature")
            .last()
            .reset_index()
        )

        # Join tables and get rid of extraneous columns from metadata.
        df = pd.merge(
            df,
            self.friktion_metadata,
            how="left",
            left_on=tx_merge_key,
            right_on=meta_merge_key,
            suffixes=("", "_drop"),
        )
        df.drop([col for col in df.columns if "drop" in col], axis=1, inplace=True)

        # Tag the row with the instructionType
        df["userAction"] = instructionType

        df.drop_duplicates().to_csv(output_file, index=False)
        print(datetime.now(), "{} data size: {}".format(instructionType, df.shape[0]))

    def parse_all(self):
        if not self.skip_ix_scrape:
            self.get_ix_batch()
        self.parse_claim_withdrawal()
        self.parse_deposit_cancel()
        self.parse_withdrawal_cancel()
        self.parse_deposits()
        self.parse_withdrawal()

    ########################################################################################################
    #####################################          Tests             #######################################
    ########################################################################################################

    def check_fidelity(self):
        ix = pd.read_csv(self.ix_fname)
        deposits = pd.read_csv(self.deposit_fname)
        withdrawals = pd.read_csv(self.withdraw_fname)
        claim = pd.read_csv(self.withdraw_claim_fname)
        withdrawals_cxl = pd.read_csv(self.withdraw_cxl_fname)
        deposits_cxl = pd.read_csv(self.deposit_cxl_fname)

        assert ix.loc[ix.instructionType == "Deposit"].shape[0] == deposits.shape[0]
        assert ix.loc[ix.instructionType == "Withdraw"].shape[0] == withdrawals.shape[0]
        assert (
            ix.loc[ix.instructionType == "ClaimPendingWithdrawal"].shape[0]
            == claim.shape[0]
        )
        assert (
            ix.loc[ix.instructionType == "CancelPendingDeposit"].shape[0]
            == deposits_cxl.shape[0]
        )
        assert (
            ix.loc[ix.instructionType == "CancelPendingWithdrawal"].shape[0]
            == withdrawals_cxl.shape[0]
        )


# if __name__ == "__main__":
#     date_start = "2021-12-16T16:00:00Z"
#     date_end = "2022-04-01T00:00:00Z"

#     x = MyPortfolio(date_start, date_end)
#     x.parse_all()
#     x.check_fidelity()


In [124]:
date_start = "2021-12-16T00:00:00Z"
date_end = "2022-04-02T00:00:00Z"

x = MyPortfolio(date_start, date_end, skip_ix_scrape=True)

x.parse_all()

x.check_fidelity()

2022-03-31 20:19:42.106867 Parsing transfers for instructionType: ClaimPendingWithdrawal
2022-03-31 20:19:42.117921 64 signature batches required...
2022-03-31 20:19:45.282583 213 transfers scraped in batch 0
2022-03-31 20:19:48.158102 163 transfers scraped in batch 1
2022-03-31 20:19:51.051203 183 transfers scraped in batch 2
2022-03-31 20:19:53.959187 244 transfers scraped in batch 3
2022-03-31 20:19:56.920423 171 transfers scraped in batch 4
2022-03-31 20:19:59.809382 221 transfers scraped in batch 5
2022-03-31 20:20:04.497262 219 transfers scraped in batch 6
2022-03-31 20:20:10.328160 241 transfers scraped in batch 7
2022-03-31 20:20:15.717891 191 transfers scraped in batch 8
2022-03-31 20:20:19.342960 204 transfers scraped in batch 9
2022-03-31 20:20:22.110222 188 transfers scraped in batch 10
2022-03-31 20:20:25.748282 223 transfers scraped in batch 11
2022-03-31 20:20:29.427290 225 transfers scraped in batch 12
2022-03-31 20:20:34.989326 248 transfers scraped in batch 13
2022-03

2022-03-31 20:26:37.447905 313 transfers scraped in batch 38
2022-03-31 20:26:40.339587 306 transfers scraped in batch 39
2022-03-31 20:26:43.293607 309 transfers scraped in batch 40
2022-03-31 20:26:46.120624 340 transfers scraped in batch 41
2022-03-31 20:26:49.049887 317 transfers scraped in batch 42
2022-03-31 20:26:51.897432 324 transfers scraped in batch 43
2022-03-31 20:26:54.791895 335 transfers scraped in batch 44
2022-03-31 20:26:57.705566 357 transfers scraped in batch 45
2022-03-31 20:27:00.550214 325 transfers scraped in batch 46
2022-03-31 20:27:03.437803 319 transfers scraped in batch 47
2022-03-31 20:27:06.306769 294 transfers scraped in batch 48
2022-03-31 20:27:14.867841 304 transfers scraped in batch 49
2022-03-31 20:27:19.235122 312 transfers scraped in batch 50
2022-03-31 20:27:22.417206 361 transfers scraped in batch 51
2022-03-31 20:27:29.285043 265 transfers scraped in batch 52
2022-03-31 20:27:32.582905 217 transfers scraped in batch 53
2022-03-31 20:27:36.2976

2022-03-31 20:34:36.294031 325 transfers scraped in batch 172
2022-03-31 20:34:39.058792 344 transfers scraped in batch 173
2022-03-31 20:34:42.140181 464 transfers scraped in batch 174
2022-03-31 20:34:44.994530 394 transfers scraped in batch 175
2022-03-31 20:34:48.198881 420 transfers scraped in batch 176
2022-03-31 20:34:51.344878 489 transfers scraped in batch 177
2022-03-31 20:34:54.187634 369 transfers scraped in batch 178
2022-03-31 20:34:57.019934 333 transfers scraped in batch 179
2022-03-31 20:35:00.100884 423 transfers scraped in batch 180
2022-03-31 20:35:03.264573 429 transfers scraped in batch 181
2022-03-31 20:35:06.318468 436 transfers scraped in batch 182
2022-03-31 20:35:14.091200 430 transfers scraped in batch 183
2022-03-31 20:35:17.703155 415 transfers scraped in batch 184
2022-03-31 20:35:23.515947 442 transfers scraped in batch 185
2022-03-31 20:35:27.585530 454 transfers scraped in batch 186
2022-03-31 20:35:31.525281 333 transfers scraped in batch 187
2022-03-

2022-03-31 20:42:55.639083 300 transfers scraped in batch 305
2022-03-31 20:42:58.815470 273 transfers scraped in batch 306
2022-03-31 20:43:02.802956 288 transfers scraped in batch 307
2022-03-31 20:43:06.232716 284 transfers scraped in batch 308
2022-03-31 20:43:09.265761 281 transfers scraped in batch 309
2022-03-31 20:43:12.178701 291 transfers scraped in batch 310
2022-03-31 20:43:15.052396 276 transfers scraped in batch 311
2022-03-31 20:43:18.920343 268 transfers scraped in batch 312
2022-03-31 20:43:22.646574 264 transfers scraped in batch 313
2022-03-31 20:43:26.094170 284 transfers scraped in batch 314
2022-03-31 20:43:28.962818 276 transfers scraped in batch 315
2022-03-31 20:43:31.624895 268 transfers scraped in batch 316
2022-03-31 20:43:34.399776 290 transfers scraped in batch 317
2022-03-31 20:43:37.509416 260 transfers scraped in batch 318
2022-03-31 20:43:40.345134 280 transfers scraped in batch 319
2022-03-31 20:43:43.158456 284 transfers scraped in batch 320
2022-03-

AssertionError: 

In [128]:
        ix = pd.read_csv("friktion_ix.csv")
        deposits = pd.read_csv("friktion_deposit.csv")
#         withdrawals = pd.read_csv(self.withdraw_fname)
#         claim = pd.read_csv(self.withdraw_claim_fname)
#         withdrawals_cxl = pd.read_csv(self.withdraw_cxl_fname)
#         deposits_cxl = pd.read_csv(self.deposit_cxl_fname)
        
#         print(ix)
#         print(deposits)

#         assert ix.loc[ix.instructionType=="Deposit"].shape[0] == deposits.shape[0]
#         assert ix.loc[ix.instructionType=="Withdraw"].shape[0] == withdrawals.shape[0]
#         assert ix.loc[ix.instructionType=="ClaimPendingWithdrawal"].shape[0] == claim.shape[0]
#         assert ix.loc[ix.instructionType=="CancelPendingDeposit"].shape[0] == deposits_cxl.shape[0]
#         assert ix.loc[ix.instructionType=="CancelPendingWithdrawal"].shape[0] == withdrawals_cxl.shape[0]

In [129]:
deposits.shape

(29406, 17)

In [131]:
ix.loc[ix.instructionType=="Deposit"].shape

(29408, 5)

In [111]:
deposits.columns

Index(['txSignature', 'amount', 'instructionAction', 'instructionOrder',
       'userAddress', 'timestamp', 'currencyName', 'currencyAddress',
       'senderAddress', 'senderTokenMint', 'receiverAddress', 'globalId',
       'vaultAuthority', 'shareTokenMint', 'depositTokenSymbol',
       'depositTokenCoingeckoId', 'userAction'],
      dtype='object')

In [ ]:
deposits

In [28]:
# Have Price
# Have VoltTokenShare

In [27]:
deposits.columns

Index(['txSignature', 'amount', 'instructionAction', 'instructionOrder',
       'userAddress', 'timestamp', 'currencyName', 'currencyAddress',
       'senderAddress', 'senderTokenMint', 'receiverAddress', 'globalId',
       'vaultAuthority', 'shareTokenMint', 'depositTokenSymbol',
       'depositTokenCoingeckoId'],
      dtype='object')

In [16]:
deposits.shape

(115, 16)

In [26]:
deposits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   txSignature              115 non-null    object 
 1   amount                   115 non-null    float64
 2   instructionAction        115 non-null    object 
 3   instructionOrder         115 non-null    object 
 4   userAddress              115 non-null    object 
 5   timestamp                115 non-null    object 
 6   currencyName             115 non-null    object 
 7   currencyAddress          115 non-null    object 
 8   senderAddress            115 non-null    object 
 9   senderTokenMint          115 non-null    object 
 10  receiverAddress          115 non-null    object 
 11  globalId                 115 non-null    object 
 12  vaultAuthority           115 non-null    object 
 13  shareTokenMint           115 non-null    object 
 14  depositTokenSymbol       1

In [17]:
deposits.columns

Index(['txSignature', 'amount', 'instructionAction', 'instructionOrder',
       'userAddress', 'timestamp', 'currencyName', 'currencyAddress',
       'senderAddress', 'senderTokenMint', 'receiverAddress', 'globalId',
       'vaultAuthority', 'shareTokenMint', 'depositTokenSymbol',
       'depositTokenCoingeckoId'],
      dtype='object')

In [18]:
deposits

,txSignature,amount,instructionAction,instructionOrder,userAddress,timestamp,currencyName,currencyAddress,senderAddress,senderTokenMint,receiverAddress,globalId,vaultAuthority,shareTokenMint,depositTokenSymbol,depositTokenCoingeckoId
0,1kx6PYTANDwsAZGZ5xoRNe51uRK68k6roF6KYDauHQMmem...,0.998590,transfer,1-1,79jbRVMQu5rQkEcAPbzZNqnwKEASHyEyWgUn9q76yUKT,2022-03-17T19:33:26Z,Wrapped Ethereum (Sollet),2FPyTwcZLUg1MDrwsyoP4D6s1tM7hAkHYRjkNb5w6Pxk,79jbRVMQu5rQkEcAPbzZNqnwKEASHyEyWgUn9q76yUKT,3e81Tr2AGVLwPLLsUwiBSaYdGDwSxCMXmhF5ZWUuQAsn,FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp,mainnet_income_call_eth,FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp,GjnoPUjQiEUYWuKAbMax2cM1Eony8Yutc133wuSun9hS,ETH,ethereum
1,231BLTQGbub2Cdm7s5Y7rSS4JN2L9JLWzhtwYG7Jowpd3e...,144.809361,transfer,1-1,EapjtDx27xLxAeKFhFUzsYdCT6BeeAEKcJ4NYoMqKubN,2022-03-17T18:19:02Z,USD Coin,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,EapjtDx27xLxAeKFhFUzsYdCT6BeeAEKcJ4NYoMqKubN,AaV2HTiE7hJWzDA1od8wbBAGM1Kqmxo8BSoMkGV5VKbG,GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f,mainnet_income_put_btc,GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f,THjfJ7GUeW6aMU6dzYYFVs5LnKNvmPzgk2wbh3bWagC,USDC,usd-coin
2,24y4KRma4BUarjw4y8Rv9Dt1kZdVVSP5kVnkbu4Ra5KMs7...,551.000000,transfer,3-1,9T19SkKUpSNbt2cMcCQVz7NZEfmXW3gdMDkH9db1yidw,2022-03-17T18:39:38Z,Wrapped SOL,So11111111111111111111111111111111111111112,9T19SkKUpSNbt2cMcCQVz7NZEfmXW3gdMDkH9db1yidw,D9QnLjbjTEBCqFZUrovoFkNW3B8gLmsJkT6PDt4HUv1H,wJAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf,mainnet_income_call_sol_high,wJAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,SOL,solana
3,28SouEpsyqsi3u6Mmy1XRbjCEaab4M6YtofuEK1c85CDZv...,100.000000,transfer,1-1,5KfRm2tAzQ4yFWKVvbWdtesvz4iuBfPxdJYwZB1yNJHP,2022-03-17T22:51:42Z,USD Coin,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,5KfRm2tAzQ4yFWKVvbWdtesvz4iuBfPxdJYwZB1yNJHP,DMEMt2oVMq8TRvBWY79xfgYb6Kq5GHcwR8cKHgDy934S,GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f,mainnet_income_put_btc,GrB6vbG2WP7eEnbwgxUbBGRMeXYq139jo2o9oW8cNK8f,THjfJ7GUeW6aMU6dzYYFVs5LnKNvmPzgk2wbh3bWagC,USDC,usd-coin
4,28nZcxYZ9sMCbotCXRrdRLx8GWksnY4PeJYAeiaRcKNyGk...,0.388000,transfer,2-0,D6mQT7imPfYF4VUVdVXKR1Rgya6oefmxYnihzy9CpUxB,2022-03-17T21:20:45Z,Wrapped SOL,So11111111111111111111111111111111111111112,D6mQT7imPfYF4VUVdVXKR1Rgya6oefmxYnihzy9CpUxB,25rEKbr8KDHj3UT1ZSSBYsmDCUQnWXn6PEFJMMZ6bHdg,wJAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf,mainnet_income_call_sol_high,wJAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf,DNa849drqW19uBV5X9ohpJ5brRGzq856gk3HDRqveFrA,SOL,solana
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,fYVnATWfXi5r16jP1VPgSd7ppUoZKkrSgKEDEo8FJ4WhZh...,10000.000000,transfer,0-0,AT4qoJDVGXpEm3BGACUhC1zpVNLq4iX2bJ4cpABehpoD,2022-03-17T23:32:03Z,USD Coin,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,AT4qoJDVGXpEm3BGACUhC1zpVNLq4iX2bJ4cpABehpoD,347NS5vxWV3p7GCHT7jgNDpsKHXzLYghro9BJxuXTpNk,BThMeTgWZBoBbAzp9sK9T7gQzpQDQdRQUVLtVQ3781q1,mainnet_income_put_sol_high,BThMeTgWZBoBbAzp9sK9T7gQzpQDQdRQUVLtVQ3781q1,G8jsAWUA2KdDn7XmV1sBqUdbEXESaPdjPWDEYCsnkRX2,USDC,usd-coin
111,mT2cRegqHKyJ9wGUV4Eu5UPX5sK1csJBfFTv3cPiNMkkAD...,145.317189,transfer,1-1,GedG3v5kyfKaqFdUXpY9SJ8tvZaKY1em5TrUf9ENfaPt,2022-03-17T21:14:53Z,Wrapped FTT (Sollet),AGFEad2et2ZJif9jaGpdMixQqvW5i81aBdvKe7PHNfz3,GedG3v5kyfKaqFdUXpY9SJ8tvZaKY1em5TrUf9ENfaPt,FCFaWp8pyVG3iJA7YrDKC8nri9aLKz3BgVoKqCoKgwv7,7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K,mainnet_income_call_ftt,7KqHFuUksvNhrWgoacKkqyp2RwfBNdypCYgK9nxD1d6K,7wDh4VCTPwx41kvbLE6fkFgMEjnqw7NpGJvQtNabCm2B,FTT,ftx-token
112,uaZuopPrzfkJctoVVHoqz7SgP8P6uqu6wvMB63iAzmDL3t...,0.200000,transfer,3-0,D2Mo1W5UDAZJaKTZVp9QpaZGxXMJx58joijb9UrYcUNv,2022-03-17T18:42:51Z,Wrapped SOL,So11111111111111111111111111111111111111112,D2Mo1W5UDAZJaKTZVp9QpaZGxXMJx58joijb9UrYcUNv,9funBx9gwNU1PBvNkZVL1iohS4oEBYrKXaUuLgsdaAjp,Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G,mainnet_income_call_sol,Hxtb6APfNtf9m8jJjh7uYp8fCTGr9aeHxBSfiPqCrV6G,4Hnh1UCC6HLzx9NaGKnTVHR2bANcRrhydumdHCnrT3i2,SOL,

In [19]:
deposits.loc[deposits.depositTokenCoingeckoId.isna()].currencyAddress

Series([], Name: currencyAddress, dtype: object)

In [20]:
deposits.shape

(115, 16)

In [21]:
deposits.loc[deposits.txSignature=="24y4KRma4BUarjw4y8Rv9Dt1kZdVVSP5kVnkbu4Ra5KMs7G9Df2Vb9C1b3M8f6yCuNuhsEW9s1WR8WrSKuN5bVBX"].iloc[0]

txSignature                24y4KRma4BUarjw4y8Rv9Dt1kZdVVSP5kVnkbu4Ra5KMs7...
amount                                                                 551.0
instructionAction                                                   transfer
instructionOrder                                                         3-1
userAddress                     9T19SkKUpSNbt2cMcCQVz7NZEfmXW3gdMDkH9db1yidw
timestamp                                               2022-03-17T18:39:38Z
currencyName                                                     Wrapped SOL
currencyAddress                  So11111111111111111111111111111111111111112
senderAddress                   9T19SkKUpSNbt2cMcCQVz7NZEfmXW3gdMDkH9db1yidw
senderTokenMint                 D9QnLjbjTEBCqFZUrovoFkNW3B8gLmsJkT6PDt4HUv1H
receiverAddress                  wJAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf
globalId                                        mainnet_income_call_sol_high
vaultAuthority                   wJAoeEG2sfQ1xgXUNVVkJ5mCTCw4SLc6oJafDwf6jTf

In [22]:
deposits.loc[deposits.txSignature=="1kx6PYTANDwsAZGZ5xoRNe51uRK68k6roF6KYDauHQMmemH4JunDBCA47oHT2J1QUbh5YJHFdoZtQECqUVLVbZp"].iloc[0]

txSignature                1kx6PYTANDwsAZGZ5xoRNe51uRK68k6roF6KYDauHQMmem...
amount                                                               0.99859
instructionAction                                                   transfer
instructionOrder                                                         1-1
userAddress                     79jbRVMQu5rQkEcAPbzZNqnwKEASHyEyWgUn9q76yUKT
timestamp                                               2022-03-17T19:33:26Z
currencyName                                       Wrapped Ethereum (Sollet)
currencyAddress                 2FPyTwcZLUg1MDrwsyoP4D6s1tM7hAkHYRjkNb5w6Pxk
senderAddress                   79jbRVMQu5rQkEcAPbzZNqnwKEASHyEyWgUn9q76yUKT
senderTokenMint                 3e81Tr2AGVLwPLLsUwiBSaYdGDwSxCMXmhF5ZWUuQAsn
receiverAddress                 FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp
globalId                                             mainnet_income_call_eth
vaultAuthority                  FThcy5XXvab5u3jbA6NjWKdMNiCSV3oY5AAkvEvpa8wp